In [60]:
#--Dependencies--#

#-- Data Cleaning Libraries:
import pandas as pd
import numpy as np

#-- Data Visualization Libraries:
from matplotlib import pyplot as plt
import seaborn as sns #--just in case we need it, probably won't


#-- Web Scraping Libraries:
import os
import time
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser


In [121]:
#Settings for accessing Website

executable_path = {"executable_path": "chromedriver.exe"}

browser = Browser ('chrome', **executable_path, headless=False)



In [337]:
#Create Empty List for years we want to scrape

years = ["1990", "1991", "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999", "2000",\
        "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", \
        "2012", "2013", "2014", "2015"]
years[0:3]

['1990', '1991', '1992']

In [326]:
#URL's for both NBA and NCAA

NBA_game_url = f"https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"

NCAA_game_url = f"https://www.sports-reference.com/cbb/play-index/psl_finder.cgi?request=1&match=single&year_min={year}&class_is_fr=Y&class_is_so=Y&class_is_jr=Y&class_is_sr=Y&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&games_type=A&c1stat=pf&c1comp=gt&order_by=year_id&order_by_asc=Y"


In [266]:
#Create a url_list
url_list = [NBA_url, NCAA_url]

# browser.visit(url_list[0])



In [358]:
#Create empty DataFrames to merge player data (Stats per game), also create partial reference variable for NBA

NBA_game_df = pd.DataFrame()
NCAA_game_df = pd.DataFrame()

NBA_refer = "basketball-reference"



In [286]:
#Equation for calculating PER

PER_calc = 0

In [321]:
#Define function for cleaning and merging data (to be called in function for scraping: scrape_nba_data) 

#Function for cleaning NBA data

def clean_nba_data (page_html):
    #Use pandas to read html
    year_html = pd.read_html(html, header = 0)

    #Get the second Table with all of the Data
    year_html = year_html[1]

    #Convert into DataFrame
    year_df = pd.DataFrame(year_html)

    #Delete Column of Ranking: "Rk"
    year_df = year_df.rename(columns=({"Rk" : "Year"}))

    #Apply the year to the Year column for each row
    year_df["Year"] = year_df["Year"].apply(lambda x: year)
                
    #Calculate PER

    print(year_df.head())
                
    #Reference global variable: NBA_game_df
    global NBA_game_df

    #Append to main DataFrame: NBA_game_df
    if NBA_game_df.empty: 
        NBA_game_df = year_df
    else: 
        NBA_game_df.append(year_df, ignore_index = True)
        
    


In [364]:
#Define our function for data scraping

def scrape_nba_data(web_urls):

    for url in web_urls:
        for year in years[0:2]:
        
            #Condition to check if the Web page is for the NBA 
            if NBA_refer in url:
                #Test print url to confirm what year we're looking for
                #print(f"NBA:{url}")

                #Visit the NBA Web page
                browser.visit(url)
                #Retrieve the html for the web page
                html = browser.html
                
                #Use pandas to read html
                year_html = pd.read_html(html, header = 0)

                #Get the second Table with all of the Data
                year_html = year_html[1]

                #Convert into DataFrame
                year_df = pd.DataFrame(year_html)

                #Delete Column of Ranking: "Rk"
                year_df = year_df.rename(columns=({"Rk" : "Year"}))

                #Apply the year to the Year column for each row
                year_df["Year"] = year_df["Year"].apply(lambda x: year)
                
                #Calculate PER

                print(year_df.head())
                
                #Reference global variable: NBA_game_df
                global NBA_game_df

                #Append to main DataFrame: NBA_game_df
                if NBA_game_df.empty: 
                    NBA_game_df = year_df
                else: 
                    NBA_game_df = NBA_game_df.append(year_df, ignore_index = True)
                
                
            
        #hoooldd on wait a second, let me put some sleep in it
        time.sleep(1)
            
#         #Visit NCAA url
#         else:
#             print(f"NCAA:{url}")
#             browser.visit(url)
#             html = browser.html


In [365]:
#Scrape and clean data for NBA

scrape_nba_data(url_list)



   Year         Player Pos Age   Tm   G  GS    MP   FG   FGA  ...    FT%  ORB  \
0  1990     Mark Acres   C  27  ORL  80  50  21.1  1.7   3.6  ...   .692  1.9   
1  1990  Michael Adams  PG  27  DEN  79  74  34.1  5.0  12.5  ...   .850  0.6   
2  1990   Mark Aguirre  SF  30  DET  78  40  25.7  5.6  11.5  ...   .756  1.5   
3  1990    Danny Ainge  PG  30  SAC  75  68  36.4  6.7  15.4  ...   .831  0.9   
4  1990    Mark Alarie  PF  26  WSB  82  10  23.1  4.5   9.6  ...   .812  1.8   

   DRB  TRB  AST  STL  BLK  TOV   PF   PTS  
0  3.5  5.4  0.8  0.5  0.3  0.9  3.1   4.5  
1  2.2  2.8  6.3  1.5  0.0  1.8  1.7  15.5  
2  2.4  3.9  1.9  0.4  0.2  1.6  2.6  14.1  
3  3.4  4.3  6.0  1.5  0.2  2.5  3.2  17.9  
4  2.7  4.6  1.7  0.7  0.5  1.2  2.7  10.5  

[5 rows x 30 columns]
   Year         Player Pos Age   Tm   G  GS    MP   FG   FGA  ...    FT%  ORB  \
0  1991     Mark Acres   C  27  ORL  80  50  21.1  1.7   3.6  ...   .692  1.9   
1  1991  Michael Adams  PG  27  DEN  79  74  34.1  5.0  12

In [366]:
#Test print DataFrames to make sure we have the right values

NBA_game_df

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1990,Mark Acres,C,27,ORL,80,50,21.1,1.7,3.6,...,.692,1.9,3.5,5.4,0.8,0.5,0.3,0.9,3.1,4.5
1,1990,Michael Adams,PG,27,DEN,79,74,34.1,5.0,12.5,...,.850,0.6,2.2,2.8,6.3,1.5,0.0,1.8,1.7,15.5
2,1990,Mark Aguirre,SF,30,DET,78,40,25.7,5.6,11.5,...,.756,1.5,2.4,3.9,1.9,0.4,0.2,1.6,2.6,14.1
3,1990,Danny Ainge,PG,30,SAC,75,68,36.4,6.7,15.4,...,.831,0.9,3.4,4.3,6.0,1.5,0.2,2.5,3.2,17.9
4,1990,Mark Alarie,PF,26,WSB,82,10,23.1,4.5,9.6,...,.812,1.8,2.7,4.6,1.7,0.7,0.5,1.2,2.7,10.5
5,1990,Steve Alford,PG,25,DAL,41,0,7.4,1.5,3.4,...,.946,0.0,0.6,0.6,1.0,0.4,0.1,0.4,0.5,4.1
6,1990,Randy Allen,SG,25,SAC,63,6,11.8,1.7,3.8,...,.535,0.8,1.4,2.2,0.4,0.3,0.3,0.4,1.6,3.7
7,1990,Greg Anderson,PF,25,MIL,60,28,21.5,3.7,7.2,...,.535,1.9,4.4,6.2,0.4,0.5,0.9,1.3,2.9,8.8
8,1990,Nick Anderson,SG,22,ORL,81,9,22.0,4.6,9.3,...,.705,1.3,2.6,3.9,1.5,0.9,0.4,1.7,1.7,11.5
9,1990,Richard Anderson,PF,29,CHH,54,2,11.2,1.6,3.9,...,.783,0.6,1.7,2.4,1.0,0.4,0.2,0.5,1.2,4.3
